<a href="https://colab.research.google.com/github/mostume222/inf-test/blob/master/infa_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 0.** If for some reason you encounter issues navigating the folders in the project or having errors you can delete the project folder in the online colab environment by using the following code cell. If you are just starting to use this fresh colab notebook skip this step.

In [1]:
!cd ~
!pwd
!ls
!rm -r inf-test/

/content
sample_data
rm: cannot remove 'inf-test/': No such file or directory


**Step 1.** Clone github repository from link https://github.com/mostume222/inf-test.git , this will download the source code and also make the correct project structure to tun the compression program.

In [1]:
!git clone https://github.com/mostume222/inf-test.git
%cd inf-test
!git checkout master #go to master branch to have the correct project structure
!ls

Cloning into 'inf-test'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 126 (delta 42), reused 83 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (126/126), 16.07 MiB | 4.26 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/content/inf-test
Branch 'master' set up to track remote branch 'master' from 'origin'.
Switched to a new branch 'master'
data  env  infa_main.ipynb  output_compression	README.md  sequences  src


**Step 2.** Install the conda environment in /inf-test/env/infa.yml to install dependencies neded to run the compression procedure.

If you would like to install your own dependencies, the important ones are:

* cd-hit 4.8.1  
* bowtie2 2.5.4
* cutadapt 3.5
* python 3.6.13
* perl 5.34.0
* pip 21.2.2

In [2]:
#install condacolab
!ls
!pip install -q condacolab
import condacolab
condacolab.install()
!ls
#create conda envionment in infa.yml file
!conda env create -f env/infa.yml --quiet
!conda env list

data  env  infa_main.ipynb  output_compression	README.md  sequences  src
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...
condacolab_install.log	data  env  infa_main.ipynb  output_compression	README.md  sequences  src
Channels:
 - defaults
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: ...working... done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
# conda environments:
#
base                     /usr/local
infc                     /usr/local/envs/infc



**Step 3.** Run the following cell to run the compression procedure to generate the pangenome sequences of the input sequence.*italicized text*

In [1]:
#  run the script with conda env
%cd /content/inf-test/src/compressor/
!source activate infc && ls
!chmod +x run_compress.sh
!source activate infc && ./run_compress.sh 0.94 0.958 200 50 100000 1,.comp


Streaming output truncated to the last 5000 lines.
current genome kmer 7538 25
current genome kmer 7538 30
current genome kmer 7538 31
current genome kmer 7538 32
current genome kmer 7538 33
current genome kmer 7538 34
current genome kmer 7538 35
current genome kmer 7538 36
current genome kmer 7538 37
current genome kmer 7538 38
current genome kmer 7538 39
current genome kmer 7538 40
current genome kmer 7541 2
current genome kmer 7541 4
current genome kmer 7541 5
current genome kmer 7541 6
current genome kmer 7541 10
current genome kmer 7541 11
current genome kmer 7541 12
current genome kmer 7541 14
current genome kmer 7541 15
current genome kmer 7541 17
current genome kmer 7541 18
current genome kmer 7544 1
current genome kmer 7544 2
current genome kmer 7544 3
current genome kmer 7544 4
current genome kmer 7544 5
current genome kmer 7544 6
current genome kmer 7544 7
current genome kmer 7544 8
current genome kmer 7544 9
current genome kmer 7544 10
current genome kmer 7544 11
current ge

In [ ]:
#clean
!rm -r /content/inf-test/output_compression/1,.comp/

rm: cannot remove '/content/inf-test/output_compression/1,.comp/': No such file or directory
